In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
import os
for dirname, _, filenames in os.walk('.\input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
# Directory with our training horse pictures
a_dir = os.path.join('./input/MangoFruitDDS/SenMangoFruitDDS_original/Alternaria')

# Directory with our training human pictures
b_dir = os.path.join('./input/MangoFruitDDS/SenMangoFruitDDS_original/Anthracnose')

# Directory with our training horse pictures
c_dir = os.path.join('./input/MangoFruitDDS/SenMangoFruitDDS_original/Black Mould Rot')

# Directory with our training human pictures
d_dir = os.path.join('./input/MangoFruitDDS/SenMangoFruitDDS_original/Healthy')

# Directory with our training horse pictures
e_dir = os.path.join('./input/MangoFruitDDS/SenMangoFruitDDS_original/Stem end Rot')

In [ ]:
a_names = os.listdir(a_dir)
print(a_names[:10])
b_names = os.listdir(b_dir)
print(b_names[:10])
c_names = os.listdir(c_dir)
print(c_names[:10])
d_names = os.listdir(d_dir)
print(d_names[:10])
e_names = os.listdir(e_dir)
print(e_names[:10])

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 4x4 configuration
nrows = 10
ncols = 4

# Index for iterating over images
pic_index = 0